In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
from tqdm.autonotebook import tqdm

/Users/sumito/.local/share/virtualenvs/sumito-ejPVx-LA/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


# Read data

In [3]:
filelist = list(Path("data").glob("*.csv"))
filelist

[PosixPath('data/sales_train.csv'),
 PosixPath('data/shops.csv'),
 PosixPath('data/test.csv'),
 PosixPath('data/item_categories.csv'),
 PosixPath('data/items.csv'),
 PosixPath('data/sample_submission.csv')]

In [10]:
data = {
    "train": pd.read_csv("data/sales_train.csv", parse_dates=[0]),
    "test": pd.read_csv("data/test.csv"),
    "shops": pd.read_csv("data/shops.csv"),
    "items": pd.read_csv("data/items.csv"),
    "item_cat": pd.read_csv("data/item_categories.csv"),
    "sample": pd.read_csv("data/sample_submission.csv")
}

In [11]:
for key in data.keys():
    print(key)
    display(data[key].head())

train


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1.0
1,2013-03-01,0,25,2552,899.00,1.0
2,2013-05-01,0,25,2552,899.00,-1.0
3,2013-06-01,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


test


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


shops


...

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


item_cat


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


sample


,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
